In [ ]:
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("yolo11x-obb.pt")  # load a pretrained model (recommended for training)


results = model.train(
    data="data.yaml",
    epochs=50,
    batch=8,
    imgsz=256,
    name='ship_detection',
    device=0,
    single_cls=True,
    # patience=6,
    # visualize=True,
    multi_scale=True,
    # augment=True,
    dropout=0.1,
    # lr0=1e-4,
    # weight_decay=1e-5  # Weight Decay 줄이기
)

Ultralytics 8.3.5  Python-3.11.4 torch-2.0.1 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=obb, mode=train, model=yolo11x-obb.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=ship_detection12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=True, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pook0612 (limbw). Use `wandb login --relogin` to force relogin


Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...
AMP: checks passed 


train: Scanning D:\Workspace\SatellieteIMGDetection\datasets\labels\train.cache... 357 images, 0 backgrounds, 0 corrupt: 100%|██████████| 357/357 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning D:\Workspace\SatellieteIMGDetection\datasets\labels\val.cache... 37 images, 0 backgrounds, 0 corrupt: 100%|██████████| 37/37 [00:00<?, ?it/s]


Plotting labels to runs\obb\ship_detection12\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 173 weight(decay=0.0), 183 weight(decay=0.0005), 182 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 256 train, 256 val
Using 6 dataloader workers
Logging results to runs\obb\ship_detection12
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      4.29G        nan        nan        nan        181        224:  24%|██▍       | 11/45 [00:20<00:58,  1.71s/it]

In [1]:

# 직접 YOLO 클래스를 임포트
from ultralytics import YOLO

# YOLO 모델 초기화
model = YOLO("yolo11n-obb.pt") 
results = model.train(
    data="data.yaml",
    epochs=350,
    batch=16,
    imgsz=256,
    patience=100,
    device="0",
    lr0 = 0.001,
    lrf = 0.01,
    single_cls = True,
    augment = True,
    optimizer = 'AdamW',
    seed = 44,


)



Ultralytics 8.3.5  Python-3.11.4 torch-2.0.1 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)
engine\trainer: task=obb, mode=train, model=yolo11n-obb.pt, data=data.yaml, epochs=350, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=44, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True,

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pook0612 (limbw). Use `wandb login --relogin` to force relogin


Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...
AMP: checks passed 


train: Scanning D:\Workspace\SatellieteIMGDetection\datasets\labels\train.cache... 357 images, 0 backgrounds, 0 corrupt: 100%|██████████| 357/357 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  67%|██████▋   | 2/3 [00:00<00:00,  1.86it/s]

WARNING  Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.39it/s]


                   all         37       1653       0.62      0.276      0.363      0.166
Speed: 0.3ms preprocess, 4.8ms inference, 0.0ms loss, 5.7ms postprocess per image
Results saved to runs\obb\train4


lr/pg0,█▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg2,████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▂▄▄▄▅▄▄▄▅▆▆▆▅▅▇▇▆▇▆▆▇▇▇█▇█▇▇▇▇▇████████
metrics/mAP50-95(B),▁▂▄▄▄▄▄▄▄▅▅▅▆▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇█▇▇█████████
metrics/precision(B),▁▂▆▅▅▅▅▅▆▅▆▆▇▆▆▇▇▆▇▆▇▇▇▇███▇▇█▇███▇█████
metrics/recall(B),▂▂▃▄▆▅▁▅▂▇▆▇▆▅▆▇▆▆▆▆▆▆▆▆▇█▇▇▇▅▆▆▇█▇▇▇▇▇▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▅▅▄▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁
